In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# DUSTYBOX_multi

In [ ]:
vCOM = np.array([1.16666666666667, 1.16666666666667, 0.63963963963963])
lambda_1 = np.array([ -0.63397459621556, -141.742430504416, -0.52370200744224])
lambda_2 = np.array([ -2.36602540378444, -1058.25756949558, -105.976297992557])
cg_1 = np.array([ -0.22767090063074, -0.35610569612832, -0.06458203330249])
cg_2 = np.array([ 0.06100423396407, 0.18943902946166, 0.42494239366285])
cd_1_1 = np.array([ 0.84967936855889, 0.85310244713865, 1.36237475791577])
cd_1_2 = np.array([ -0.01634603522555, -0.01976911380532, -0.00201439755542])
cd_2_1 = np.array([ -0.62200846792815, -0.49699675101033, -0.13559165545855])
cd_2_2 = np.array([ -0.04465819873852, -0.16966991565634, -0.00404798418109])

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(21,7))
tmax = [3,0.03,0.3]
folder = ["output_A", "output_B", "output_C"]
titles = ["Test A", "Test B", "Test C"]
for j in range(0,3):
    for i in range(0,30):
        data = np.loadtxt("DUSTYBOX/%s/DHD/%d.txt"%(folder[j],i))
        axs[j].scatter(data[0,-1], np.mean(data[:,1]), color='#DC2525', marker='o', alpha=0.6, s=80)
        axs[j].scatter(data[0,-1], np.mean(data[:,4]), color='#320A6B', marker='o', alpha=0.6, s=80)
        axs[j].scatter(data[0,-1], np.mean(data[:,6]), color='#0F828C', marker='o', alpha=0.6, s=80)

    axs[j].grid(alpha=0.3)


    t = np.linspace(0,tmax[j],1000)
    itest = j
    v_g = vCOM[itest] + cg_1[itest] * np.exp(lambda_1[itest]*t) + cg_2[itest]*np.exp(lambda_2[itest]*t)
    v_d_1 = vCOM[itest] + cd_1_1[itest] * np.exp(lambda_1[itest]*t) + cd_1_2[itest]*np.exp(lambda_2[itest]*t)
    v_d_2 = vCOM[itest] + cd_2_1[itest] * np.exp(lambda_1[itest]*t) + cd_2_2[itest]*np.exp(lambda_2[itest]*t)

    axs[j].plot(t, v_g, color='#DC2525', label='Gas', lw=3)
    axs[j].plot(t, v_d_1, color='#320A6B', label='Dust - 1', lw=3)
    axs[j].plot(t, v_d_2, color='#0F828C', label='Dust - 2', lw=3)
    axs[j].set_title(titles[j], fontsize=22, pad=10)

    
    axs[j].set_xlabel("Time", fontsize=20)
    axs[j].tick_params(labelsize=16)

axs[0].set_ylabel("Velocity", fontsize=20)
axs[1].axes.yaxis.set_ticklabels([])
axs[2].axes.yaxis.set_ticklabels([])
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Numerical', markerfacecolor='black',alpha=0.8, markersize=10),
                   Line2D([0], [0], color='black', label='Analytical', lw=3),
                   Line2D([0], [0], color='#DC2525', lw=3, label='Gas'),
                   Line2D([0], [0], color='#320A6B', lw=3, label='Dust - 1'),
                   Line2D([0], [0], color='#0F828C', lw=3, label='Dust - 2')]

# Create the figure
axs[0].legend(handles=legend_elements,fontsize=16)
fig.tight_layout()

plt.savefig("DUSTYBOX.png", dpi=600, bbox_inches='tight')

# DUSTYWAVE_multi

In [ ]:
from scipy.optimize import curve_fit
def wave(x, f0, df_real, df_im):
    return f0 + 1e-4 * (df_real * np.cos(2*np.pi*x) - df_im * np.sin(2*np.pi*x))


def fit_wave(data):
    x = np.linspace(0,1,data.shape[0])
    popt, _ = curve_fit(wave, x, data, maxfev = int(1e5))
    return popt[1] / popt[0]

def fit_wave_vel(data):
    x = np.linspace(0,1,data.shape[0])
    popt, _ = curve_fit(wave, x, data, maxfev = int(1e5))
    return popt[1]


In [ ]:
fig, axs = plt.subplots(1,2,figsize=(18,7))

data = np.loadtxt("DUSTYWAVE/analytic_wave_damping_A.txt")
print(data[:,0])
axs[0].plot(data[:,0], data[:,1], label='Gas', lw=3, color='#DC2525')
axs[0].plot(data[:,0],data[:,2], label = 'Dust', lw=3, color='#065084')

for i in range(0,21):
    data = np.loadtxt("DUSTYWAVE/output_A/DHD/%d.txt"%i)
    axs[0].scatter(data[0,-1], fit_wave(data[:,0]), color='#DC2525', alpha=0.5, s=80)
    axs[0].scatter(data[0,-1], fit_wave(data[:,3]), color='#065084', alpha=0.5, s=80)

axs[0].legend(fontsize=14)
axs[0].set_ylabel("Normalized Density", fontsize=20)
axs[0].set_xlabel("Time", fontsize=20)
axs[0].grid(alpha=0.5)



data = np.loadtxt("DUSTYWAVE/analytic_wave_damping_B.txt")
print(data[:,0])
axs[1].plot(data[:,0], data[:,1], label='Gas', lw=3, color='#DC2525')
axs[1].plot(data[:,0],data[:,2], label = 'Dust - 1', lw=3, color='#78B9B5')
axs[1].plot(data[:,0],data[:,3], label = 'Dust - 2', lw=3, color='#0F828C')
axs[1].plot(data[:,0],data[:,4], label = 'Dust - 3', lw=3, color='#065084')
axs[1].plot(data[:,0],data[:,5], label = 'Dust - 4', lw=3, color='#320A6B')

for i in range(0,21):
    data = np.loadtxt("DUSTYWAVE/output_B/DHD/%d.txt"%i)
    axs[1].scatter(data[0,-1], fit_wave(data[:,0]), color='#DC2525', alpha=0.5, s=80)
    axs[1].scatter(data[0,-1], fit_wave(data[:,3]), color='#78B9B5', alpha=0.5, s=80)
    axs[1].scatter(data[0,-1], fit_wave(data[:,5]), color='#0F828C', alpha=0.5, s=80)
    axs[1].scatter(data[0,-1], fit_wave(data[:,7]), color='#065084', alpha=0.5, s=80)
    axs[1].scatter(data[0,-1], fit_wave(data[:,9]), color='#320A6B', alpha=0.5, s=80)

axs[1].legend(fontsize=14)
axs[1].set_ylabel("Normalized Density", fontsize=20)
axs[1].set_xlabel("Time", fontsize=20)
axs[1].grid(alpha=0.3)
for ax in axs:
    ax.tick_params(labelsize=18)

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Numerical', markerfacecolor='black',alpha=0.8, markersize=10),
                   Line2D([0], [0], color='black', label='Analytical', lw=3),
                   Line2D([0], [0], color='#DC2525', lw=3, label="Gas"),
                   Line2D([0], [0], color='#065084', lw=3, label= "Dust")]

# Create the figure
axs[0].legend(handles=legend_elements,fontsize=18)

legend_elements = [Line2D([0], [0], color='#DC2525', lw=3, label="Gas"),
                   Line2D([0], [0], color='#78B9B5', lw=3, label= "Dust - 1"),
                   Line2D([0], [0], color='#0F828C', lw=3, label= "Dust - 2"),
                   Line2D([0], [0], color='#065084', lw=3, label= "Dust - 3"),
                   Line2D([0], [0], color='#320A6B', lw=3, label= "Dust - 4")]

# Create the figure
axs[1].legend(handles=legend_elements,fontsize=18)

fig.tight_layout()

fig.savefig("DUSTYWAVE.png", dpi=600, bbox_inches='tight')

# DUSTYSHOCK_multi

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(18,6))

data = np.loadtxt("DUSTYSHOCK/output_B/DHD/50.txt")
x = np.linspace(0,40,400)
N_plot = 4
axs[0].scatter(x[::N_plot], data[::N_plot,0], color='#DC2525', alpha=0.5, s=50)
axs[0].scatter(x[::N_plot], data[::N_plot,3], color='#320A6B', alpha=0.5, s=50)
axs[0].scatter(x[::N_plot], data[::N_plot,5], color='#065084', alpha=0.5, s=50)
axs[0].scatter(x[::N_plot], data[::N_plot,7], color='#78B9B5', alpha=0.5, s=50)

axs[1].scatter(x[::N_plot], data[::N_plot,1] / 2, label='Gas', color='#DC2525', alpha=0.5, s=50)
axs[1].scatter(x[::N_plot], data[::N_plot,4] / 2, label='Dust - 1', color='#320A6B', alpha=0.5, s=50)
axs[1].scatter(x[::N_plot], data[::N_plot,6] / 2, label='Dust - 2', color='#065084', alpha=0.5, s=50)
axs[1].scatter(x[::N_plot], data[::N_plot,8] / 2, label='Dust - 3', color='#78B9B5', alpha=0.5, s=50)


data = np.loadtxt("DUSTYSHOCK/steady_state.txt")
data[:,0] -= 0.15
axs[0].plot(data[:,0], data[:,1], color='#DC2525', lw=3)
axs[0].plot(data[:,0], data[:,2], color='#320A6B', lw=3)
axs[0].plot(data[:,0], data[:,3], color='#065084', lw=3)
axs[0].plot(data[:,0], data[:,4], color='#78B9B5', lw=3)


axs[1].plot(data[:,0], data[:,5], color='#DC2525', lw=3)
axs[1].plot(data[:,0], data[:,6], color='#320A6B', lw=3)
axs[1].plot(data[:,0], data[:,7], color='#065084', lw=3)
axs[1].plot(data[:,0], data[:,8], color='#78B9B5', lw=3)

axs[1].set_yscale('log')
axs[0].set_xlim(0,20)
axs[0].set_ylim(0,20)
axs[1].set_xlim(0,20)
axs[0].set_ylabel("Density", fontsize=20)
axs[1].set_ylabel("Normalized velocity", fontsize=20)
axs[1].legend(fontsize=16)
for ax in axs:
    ax.grid(alpha=0.3)
    ax.set_xlabel("Position", fontsize=20)
    ax.tick_params(labelsize=18)


legend_elements = [Line2D([0], [0], marker='o', color='w', label='Numerical', markerfacecolor='black',alpha=0.8, markersize=10),
                   Line2D([0], [0], color='black', label='Analytical', lw=3),
                   Line2D([0], [0], color='#DC2525', lw=3, label="Gas"),
                   Line2D([0], [0], color='#78B9B5', lw=3, label= "Dust - 1"),
                   Line2D([0], [0], color='#0F828C', lw=3, label= "Dust - 2"),
                   Line2D([0], [0], color='#065084', lw=3, label= "Dust - 3"),
                   Line2D([0], [0], color='#320A6B', lw=3, label= "Dust - 4")]

# Create the figure
axs[1].legend(handles=legend_elements,fontsize=18)

fig.tight_layout()
fig.savefig("DUSTYSHOCK.png", dpi=600, bbox_inches='tight')

# External Force

In [ ]:
plt.figure(figsize=(12,7))
def get_sol(t):
    C = np.matrix([[0.833333, 0.145014, 0.059615, 1.79537],
                   [0.083333, -0.116011, 0.029808, 0.096204],
                   [0.083333, -0.029003, -0.089423, 0.068426]])
    lambda1 = -1.125 
    lambda2 = -0.8
    v = np.array([t, np.exp(lambda1*t), np.exp(lambda2*t),1])
    sol = np.array(C@v)[0]
    return sol 

t = np.linspace(0,10,1000)
vg = np.zeros_like(t)
vd1 = np.zeros_like(t)
vd2 = np.zeros_like(t)
i = 0
for it in t:
    sol = get_sol(it)
    vg[i] = sol[0]
    vd1[i] = sol[1] / 0.1
    vd2[i] = sol[2] / 0.1
    i += 1

plt.plot(t, np.abs(vg - vd1), lw=3, label = r"$|v_g - v_{d1}|$", color='#065084')
plt.plot(t, np.abs(vg - vd2), lw=3, label = r"$|v_g - v_{d2}|$", color='#DC2525')
plt.grid(alpha=0.3)

for i in range(0,100):
    data = np.loadtxt("EXT_FORCE/output/DHD/%d.txt"%i)
    plt.scatter(data[0,-1], np.abs(np.mean(data[:,1]) - np.mean(data[:,4])), color='#065084', marker='o', alpha=0.6, s=80)
    plt.scatter(data[0,-1], np.abs(np.mean(data[:,1]) - np.mean(data[:,6])), color='#DC2525', marker='o', alpha=0.6, s=80)

plt.ylabel("Gas-dust relative velocity", fontsize=24)
plt.xlabel("Time", fontsize=24)
plt.tick_params(labelsize=18)
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Numerical', markerfacecolor='black',alpha=0.8, markersize=10),
                   Line2D([0], [0], color='black', label='Analytical', lw=3),
                   Line2D([0], [0], color='#DC2525', lw=3, label=r"$|v_g - v_{d1}|$"),
                   Line2D([0], [0], color='#065084', lw=3, label= r"$|v_g - v_{d2}|$")]

# Create the figure
plt.legend(handles=legend_elements,fontsize=20)

plt.tight_layout()
plt.savefig("EXT_FORCE.png", dpi=600, bbox_inches='tight')


